University of Helsinki, Master's Programme in Mathematics and Statistics  
MAST32001 Computational Statistics, Autumn 2023  
Luigi Acerbi

# Week 4 exercises

## 1. MCMC convergence diagnostics (4 pts)

The file loaded below contains samples from 3 chains of Metropolis-Hastings sampling for the bimodal target 
$$\pi^*(\theta) = \exp(-\gamma (\theta^2-1)^2)$$
with $\gamma = 4$. Warm-up samples have already been removed from the saved results, so you do not need to remove warm-up samples.

1. Evaluate the $\hat{R}$-statistic defined in Sec. 8.1.1 of the course notes using the original 3 chains. Report your result in Moodle.
2. Evaluate the split-$\hat{R}$ (see Sec. 8.1.1) by splitting the chains in half to obtain 6 chains. Report your result in Moodle.
3. Compute the effective sample size (ESS) for the first chain (1000 samples), when we know the estimates $\mu = 0$, $\sigma^2 = 0.9185$ from an independent long simulation. Use the truncated sum estimate from Sec. 8.1.2 of the course notes. Report your result in Moodle.
   *Hint*:  Use Eqs. 8.1 and 8.4, for which you will have to find the upper bound $K$ of the sum as indicated.

In [6]:
%matplotlib inline
import pandas as pd
import numpy as np

samples = pd.read_csv('https://raw.githubusercontent.com/lacerbi/compstats-files/main/data/bimod_samples.txt', header=None, sep='\t').values

def R_roof(chains: list[list[int]]):
    n = len(chains[0])
    m = len(chains)

    theta_dot_dot = (1/m) * np.sum([np.mean(chains[j]) for j in range(0, m)])

    B = (n/(m-1)) * np.sum([(np.mean(chains[j]) - theta_dot_dot)**2 for j in range(0, m)])
    W = (1/(m*(n-1))) * np.sum([(chains[j] - np.mean(chains[j]))**2 for j in range(0, m)])
    var_theta = ((n - 1) / n)*W + B/n

    return np.sqrt(var_theta/W)

chains = [samples[:,0], samples[:,1], samples[:,2]]
print(f'(1) R_roof={R_roof(chains)}')

half_chains = [samples[:500,0], samples[500:,0], samples[:500,1], samples[500:,1], samples[:500,2], samples[500:,2]]
print(f'(2) Split R_roof={R_roof(half_chains)}')

##### 2 ######
mu = 0
sigma_sq = 0.9185
chain = chains[0]

def ro(k, chain):
    sum = 0
    n = len(chain) - k

    for i in range(n):
        sum += ((chain[i] - mu)*(chain[i+k] - mu))

    return (sum / n) / sigma_sq

K = 0
j = 1
while (ro(j, chain) + ro(j+1, chain) >= 0):
    K = j
    j += 2
K += 2
print(K)
M = len(chain)
ESS = M / (1 + 2*np.sum([ro(k, chains[0]) for k in range(K+1)]))
print(f'(3) ESS={ESS}')


(1) R_roof=1.025898298169984
(2) Split R_roof=1.1155720873796102
87
(3) ESS=13.04416694301861


## 2. MCMC sampling for a Bayesian regression model (6 pts)

In this task we will apply MCMC sampling to a linear regression model that predicts the birth weight ('bwt', $y_i$) for old mothers (age >= 30) as a function of the age of the mother, duration of the pregnancy and weight of the mother ('age', 'gestation', 'weight'; $\mathbf{x}_i$).

As our model we will consider a linear regression model fitted to the observations $((\mathbf{x}_1, y_1), (\mathbf{x}_2, y_2), \dots, (\mathbf{x}_n, y_n))$. The code below will load the matrix `x` (each row is one sample) and vector `y`. The values $\mathbf{x}_1, \dots, \mathbf{x}_n$ are the *input values* and $y_1, \dots, y_n$ are the *target values*. The loading code normalises all data to have zero mean and unit variance to get the regression coefficients to a shared scale.

We model the data with a linear regression model
$$ y_i = \mathbf{x}_i \boldsymbol{\beta} + \epsilon_i, $$
where $\mathbf{x}_i$ is a row vector of input values, $\boldsymbol{\beta} = (\beta_1, \beta_2, \beta_3)^T$ is a column vector of regression coefficients and $\epsilon_i$ is a Gaussian noise term $\epsilon_i \sim \mathcal{N}(0, \sigma_x^2)$.

The likelihood of the model is
$$ p(Y \mid X, \beta, \sigma_x) = \prod_{i=1}^n \mathcal{N}(y_i; \mathbf{x}_i \boldsymbol{\beta}, \sigma_x^2). $$

The regression coefficients $\beta_j$ have a hierarchical prior
$$ p(\beta_j | \sigma_{\beta}) = \mathcal{N}(\beta_j;\; 0, \sigma_{\beta}^2), \quad j = 1, 2, 3, $$
with standard deviation $\sigma_{\beta}$.
Its prior is
$$ p(\sigma_{\beta}) = \mathrm{Gamma}(k_{\beta} = 2, \theta_{\beta} = 1/2) $$
with shape $k_{\beta}=2$ and scale $\theta_{\beta} = 1/2$.
The prior of the noise standard deviation $\sigma_x$ is
$$ p(\sigma_{x}) = \mathrm{Gamma}(k_{x} = 2, \theta_{x} = 1/2) $$
with shape $k_{x}=2$ and scale $\theta_{x} = 1/2$.
As usual, we assume the prior factorizes as $p(\sigma_\beta, \sigma_x) = p(\sigma_\beta) p(\sigma_x)$.

To ensure the standard deviations are positive, express them as $\sigma_{\beta} = \exp(s_{\beta})$ and $\sigma_x = \exp(s_x)$. Remember to apply the density transformation to transform the priors over $\sigma_{\beta}$ and $\sigma_x$ to those over $s_{\beta}$ and $s_x$!

The full set of model parameters for MCMC is thus $\theta = (s_{\beta}, s_x, \beta_1, \beta_2, \beta_3)$.

Write a Metropolis-Hastings MCMC sampler to sample from the joint posterior distribution of all model parameters $\theta$, $$p(\theta \mid X, Y) = p(s_{\beta}, s_x, \beta_1, \beta_2, \beta_3 \mid X, Y).$$

1. Implement a Metropolis-Hastings MCMC sampler with proposal $q(\theta' ; \theta) = \mathcal{N}(\theta';\; \theta, 0.04^2 \mathbf{I})$, i.e. a multivariate normal proposal centred around the current state with standard deviation 0.04 for each component.
2. Run the sampler for 5000 iterations starting from each of the 4 initial points defined by the columns of `inits`. Discard the first 2500 iterations for each run as warm-up. Compute the split-$\hat{R}$ statistic for the samples of $\theta$. Report the largest split-$\hat{R}$ over different components to Moodle.
3. Check which variable is having convergence issues. Increase the proposal standard deviation for that component to 0.4 while keeping the proposals for the other components the same. Repeat the sampling as above and report the largest split-$\hat{R}$ over different components to Moodle.
4. Report the posterior mean of the log-standard deviation $s_{\beta}$ obtained from combining all the 10000 retained samples from the latter sampler to Moodle.

*Notes*: 
- Please note that all computations and results are based on the unbounded variables $s_{\beta}$ and $s_x$. For the purpose of this exercise, there is no need to transform anything back to the original space of $\sigma_{\beta}$ and $\sigma_x$.
- The [Gamma distribution](https://en.wikipedia.org/wiki/Gamma_distribution) has two common representations, one in terms of shape and scale parameters ($k$ and $\theta$) and one in terms of shape and rate parameters ($\alpha$ and $\beta$). Please check that you are using the correct one.

In [44]:
import pandas as pd
import numpy as np
import numpy.random as npr
from scipy import stats

# Load the data set
babies_full = pd.read_csv("https://raw.githubusercontent.com/lacerbi/compstats-files/main/data/babies2.txt", sep='\t')

# Pick a subset
babies = babies_full.loc[(babies_full['age']>=30).values,:]

x = babies[['age', 'gestation', 'weight']].values.astype(float)
y = babies[['bwt']].values.astype(float).squeeze()

# Remove mean from inputs
x -= np.mean(x, 0)
# Standardise input variance
x /= np.std(x, 0)

# Remove mean from outputs
y -= np.mean(y, 0)
# Standardise output variance
y /= np.std(y, 0)

inits = np.array([[ 1.67272789, -0.02134183, -0.78116796, -1.77420787],
                  [-0.6030697,   0.4464263,   0.3627991,  -0.7342916 ],
                  [-1.32685026, -0.37427079, -0.06744599,  0.21175491],
                  [ 0.87702047, -0.54171875, -0.44736686, -2.39045985],
                  [ 1.00631791, -1.34638203, -0.40343913, -0.64535026]])

def prior(beta, s_beta, s_x):
    beta_prior = np.sum(stats.norm.logpdf(beta, loc=0, scale=np.exp(s_beta)))
    
    log_sigma_beta_prior = stats.gamma.logpdf(np.exp(s_beta), a=2, scale=1/2) + s_beta
    log_sigma_x_prior = stats.gamma.logpdf(np.exp(s_x), a=2, scale=1/2) + s_x
    
    return beta_prior + log_sigma_beta_prior + log_sigma_x_prior

def likelihood(y, x, beta, sigma_x):
    return np.sum(stats.norm.logpdf(y, loc=x.dot(beta), scale=np.exp(sigma_x)))

def posterior(y, x, theta):
    s_beta, s_x, beta1, beta2, beta3 = theta
    beta = np.array([beta1, beta2, beta3])
    return likelihood(y, x, beta, s_x) + prior(beta, s_beta, s_x)

def mcmc(y, x, init, iterations, proposal_std):
    n_params = len(init)
    samples = []
    current_theta = init
    current_log_prob = posterior(y, x, current_theta)

    for i in range(iterations):
        proposed_theta = current_theta + proposal_std * np.random.randn(n_params)
        proposed_log_prob = posterior(y, x, proposed_theta)

        if proposed_log_prob > current_log_prob or np.random.rand() < np.exp(proposed_log_prob - current_log_prob):
            current_theta = proposed_theta
            current_log_prob = proposed_log_prob

        samples.append(current_theta)

    return samples

def R_roof(chains: list[list[int]]):
    n = len(chains[0])
    m = len(chains)

    theta_dot_dot = (1/m) * np.sum([np.mean(chains[j]) for j in range(0, m)])

    B = (n/(m-1)) * np.sum([(np.mean(chains[j]) - theta_dot_dot)**2 for j in range(0, m)])
    W = (1/(m*(n-1))) * np.sum([(chains[j] - np.mean(chains[j]))**2 for j in range(0, m)])
    var_theta = ((n - 1) / n)*W + B/n

    return np.sqrt(var_theta/W)

N = 5000
M = N // 2
proposal_std = 0.04
all_samples = np.array([mcmc(y, x, init, N, proposal_std) for init in inits.T])

for sampler_index in range(0, len(all_samples)):
    rhat_values = []

    for i in range(len(all_samples[sampler_index][0])):
        values = all_samples[sampler_index][:, i]

        value = R_roof(np.array([values[0: len(values)//2], values[len(values)//2:]]))
        rhat_values.append(value)
    largest_rhat = np.max(rhat_values)
    print(f"For run #{sampler_index} Largest split R_hatdor{largest_rhat}")

For run #0 Largest split R_hatdor1.0401458486032669
[1.0401458486032669, 1.0016738794696292, 1.0215633735252125, 1.0192199026840087, 1.0260489407969653]
For run #1 Largest split R_hatdor1.280500306696779
[1.280500306696779, 1.0095922298438273, 1.0030065577035818, 1.016573191348837, 1.0199503809948494]
For run #2 Largest split R_hatdor1.0409353474023757
[1.0409353474023757, 1.0044606367021454, 1.0052688694653225, 1.0033347268841275, 1.0125310476136904]
For run #3 Largest split R_hatdor1.036955147858673
[1.0018419030284185, 1.0250153987461739, 1.0349727943871203, 1.036955147858673, 1.0310491997245594]


## 3. Importance sampling (4 pts)

Develop an importance sampler for the bimodal target
$$\pi(\theta) = \sum_{i=1}^2 p_i \mathcal{N}(\theta ; \mu_i, \sigma_i^2),$$
where $p_1 = p_2 = 1/2$ and $\mu_1 = -1, \mu_2 = 1$.
Use $\mathrm{Laplace}(0, b)$ with a suitable $b$ as the proposal and evaluate the expectation $\mathbb{E}[(\theta-1)^2]$ when
1. $\sigma_1^2 = \sigma_2^2 = 0.5$.
2. $\sigma_1^2 = \sigma_2^2 = 0.1$.

The required tolerance for the answer is $\pm 0.1$.

In order to estimate the accuracy of your answer, it is recommended to run the sampler a few times and compute the standard deviation of the values you obtain. Monte Carlo error scales as $1/\sqrt{n}$ with the number of iterations $n$.

## 4. Rejection ABC for a dynamical model (6 pts)

In this task we will develop an ABC sampler for the autoregressive (AR) model:
$$ x_{t+1} = a x_t + \epsilon_t, \quad \epsilon_t \sim \mathcal{N}(0, \sigma^2). $$
Throughout the exercise, we assume $x_0 = 1$. The model has two parameters, $a$ and $\sigma$. We set priors
$$ p(a) = \mathrm{Uniform}(a;\; 0, 1), \quad p(\sigma) = \mathrm{Gamma}(\sigma;\; k_\sigma, \theta_\sigma) $$
with $k_\sigma = 8, \theta_\sigma = 1/8$. Note that we use the shape/scale parametrisation also used by NumPy and that the prior is over $\sigma$, not $\sigma^2$ (also more consistent with NumPy parametrisation).

1. Implement a function to simulate the AR process. Test your function by generating and plotting two independent realisations of length 200 with $a = 0.75$, $\sigma = 0.2$. Notice how the sequences diverge relatively quickly and are essentially independent toward the end. *Hint*: Length 200 means that the generated sequence will take values $x_0, x_1, \ldots, x_{199}$.
2. Sample 5000 sequences from the AR process of length 200 with $a = 0.75$, $\sigma = 0.2$. For each sequence, extract the last value. Compute the standard deviation of the last values across the 5000 simulated sequences and report it in Moodle.
3. For the purpose of ABC sampling, we are going to summarize the sequences with the 2-dimensional summary statistics
$$ S(X) = \left( \frac{1}{N} \sum_{i=0}^{N-1} x_i^2, \frac{1}{N-1} \sum_{i=0}^{N-2} x_i x_{i+1} \right). $$
Load the the single observed sequence $X$ (the data) with the code below and evaluate $S(X)$ on the provided sequence. Report the two summary statistics of the data in Moodle.
4. Implement an ABC sampler to infer the posterior over $a$ and $\sigma$ given the single observed sequence $X$ loaded below using the simulator implemented above and the 2-dimensional summary statistics defined above. Run your sampler to generate samples with acceptance threshold $\| S(X) - S(X^\star)\|_2 \le \epsilon$ with $\epsilon = 0.2$.
5. Report the approximate posterior means and standard deviations of $a$ and $\sigma$ to Moodle.

The required tolerance on the posterior means and standard deviations is $\pm 0.03$.

*Hints*:
- Note that for the acceptance threshold we are using the standard Euclidean metric: $\|v\|_2 = \sqrt{\sum_{i} v_i^2}$.
- The ABC sampler here can easily require tens of seconds or more when the number of samples is large, so you should be careful when increasing the number of samples. A reasonably good implementation should be able to reach the required accuracy in less than a minute.

In [ ]:
import pandas as pd

dataframe = pd.read_csv('https://raw.githubusercontent.com/lacerbi/compstats-files/main/data/ar_time_series_data.txt', header=None, sep='\t')
data = dataframe.values[:,0]